## Fetch embeddings

In [10]:
import requests

# Define your API key and endpoint
api_key = "BqTa5uSoo4YELoEawBVWr8PStdTFkesrWBz70ANwjrmMCR7l3VBzJQQJ99BBAC5RqLJXJ3w3AAAAACOGTjKX"
endpoint = "https://aistudiohubaih2639489385.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-05-15"

# The input text to be embedded
data = {
    "input": "hello word"
}

# Headers for the request
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}

# Make the POST request
response = requests.post(endpoint, headers=headers, json=data)

# Parse the response
if response.status_code == 200:
    embeddings = response.json()
    print(embeddings["data"][0]["embedding"])
else:
    print(f"Error: {response.status_code}, {response.text}")


[-0.018530931, 0.003953742, -0.02765345, -0.019128703, -0.01300154, 0.013189969, -0.010006183, 0.0019703733, -0.021584766, -0.012735141, 0.041376214, -0.0033332284, -0.035372507, -0.0011118883, -0.008303832, -0.003840035, 0.030252459, -0.0246386, 0.010597457, 0.005548883, -0.020103332, -0.015464101, 0.007264229, -0.008401295, -0.008797644, -0.01216336, 0.012280315, -0.021636747, 0.009401914, -0.032669537, -0.0065624965, 0.01069492, -0.02278031, -0.0074331644, -0.0054904055, -0.010305068, 0.0018761592, -0.015620042, 0.024287734, -0.0155810565, 0.0225464, -0.024014838, 0.0064292974, -0.0042818664, -0.02728959, 0.00053482724, 0.005308475, -0.021766696, 0.0047074542, 0.043793295, 0.031707905, 0.011734524, -0.017907169, -0.0018972763, -0.010941826, 0.015866946, -0.023754938, 0.02579516, 0.007179761, -0.025353327, 0.0034761738, 0.0054351767, -0.01289758, 0.01373576, 0.008030936, 0.0026493643, 0.0089925695, 0.013008038, -0.023274122, 0.0047172005, 0.01525618, 0.016087864, -0.021298874, -0.007

## Save embeddings to Azure Search Index

In [5]:
import requests
import json
from uuid import uuid4

# Define your Azure Search service details
search_service_name = "emtec-ai-search-001"
index_name = "index-001"
api_version = "2024-07-01" # "2021-04-30-Preview"
api_key = "VYfCXwB8g8ROE6nECnuNaCmmWcSClwOmzTzipZtI4jAzSeBe6YVD"

# Define the document to be added
document = {
    "id": str(uuid4().hex),
    "content": "This is a sample document content.",
    "url": "http://example.com",
    "filepath": "/path/to/file",
    "title": "Sample Document",
    "meta_json_string": json.dumps({"author": "John Doe", "category": "Sample"}),
    "contentVector": [0.1] * 3072
}

# Define the endpoint URL
endpoint = f"https://{search_service_name}.search.windows.net/indexes/{index_name}/docs/index?api-version={api_version}"

# Headers for the request
headers = {
    "Content-Type": "application/json",
    "api-key": api_key
}

# Make the POST request to add the document
response = requests.post(endpoint, headers=headers, json={"value": [document]})

# Check the response
if response.status_code == 200:
    print("Document added successfully.")
else:
    print(f"Error: {response.status_code}, {response.text}")

Document added successfully.


In [11]:
# Define the search query
search_query = {
    "search": "*",
    "searchFields": "content,title",
    "select": "content,title,url",
    "top": 5,
    "count": True
}

id = "b472e96df6f34eeda0baac6632e00cb6"
search_query = {
    "search": "*",
    "filter": f"id eq '{id}'",
    "count": True
}

# Define the search endpoint URL
search_endpoint = f"https://{search_service_name}.search.windows.net/indexes/{index_name}/docs/search?api-version={api_version}"

# Make the POST request to query the search index
search_response = requests.post(search_endpoint, headers=headers, json=search_query)

# Check the response
if search_response.status_code == 200:
    search_results = search_response.json()
    print(json.dumps(search_results, indent=2))  # Pretty print the JSON response
else:
    print(f"Error: {search_response.status_code}, {search_response.text}")

Error: 400, {"error":{"code":"","message":"Invalid expression: 'id' is not a filterable field. Only filterable fields can be used in filter expressions.\r\nParameter name: $filter"}}


## Request ChatGPT

In [13]:
import os
import base64
import requests
import json

# Define your API key and endpoint
api_key = os.getenv("AZURE_OPENAI_API_KEY", "9CX88ExgRutVwbtVazx0qTSlsIQ0PDvAGouiTOXSwyMabi9jm20DJQQJ99BBACYeBjFXJ3w3AAAAACOGrd8f")
endpoint = os.getenv("ENDPOINT_URL", "https://enric-m6qt35n1-eastus.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = "2024-05-01-preview"

# Complete API URL
api_url = f"{endpoint}openai/deployments/{deployment}/chat/completions?api-version={api_version}"

# Prepare the chat prompt
chat_prompt = [
    {"role": "system", "content": [{"type": "text", "text": "You are an AI assistant that helps people find information."}]},
    {"role": "user", "content": [{"type": "text", "text": "Give me a very short definition of artificial intelligence."}]},
]

# Request payload
data = {
    "model": deployment,
    "messages": chat_prompt,
    "max_tokens": 800,
    "temperature": 0.7,
    "top_p": 0.95,
    "frequency_penalty": 0,
    "presence_penalty": 0,
}

# Headers
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,  # Use 'api-key' instead of 'Authorization'
}

# Make the POST request
response = requests.post(api_url, headers=headers, json=data)

# Check response
if response.status_code == 200:
    print(json.dumps(response.json()["choices"][0]["message"]["content"], indent=2))  # Pretty print the JSON response
else:
    print(f"Error: {response.status_code}, {response.text}")


"Artificial intelligence (AI) is the simulation of human intelligence in machines that are programmed to think, learn, and solve problems."


In [45]:
import os
import requests
import json

# Define your API key and endpoint
api_key = os.getenv("AZURE_OPENAI_API_KEY", "9CX88ExgRutVwbtVazx0qTSlsIQ0PDvAGouiTOXSwyMabi9jm20DJQQJ99BBACYeBjFXJ3w3AAAAACOGrd8f")
endpoint = os.getenv("ENDPOINT_URL", "https://enric-m6qt35n1-eastus.openai.azure.com/")
deployment = os.getenv("DEPLOYMENT_NAME", "gpt-4o")
api_version = "2024-05-01-preview"

# Complete API URL
api_url = f"{endpoint}openai/deployments/{deployment}/chat/completions?api-version={api_version}"

# Prepare the chat prompt
chat_prompt = [
    {"role": "system", "content": [{"type": "text", "text": "You are an AI assistant that helps people find information."}]},
    {"role": "user", "content": [{"type": "text", "text": "Give me a very short definition of artificial intelligence."}]},
]

# Request payload (Enable streaming)
data = {
    "model": deployment,
    "messages": chat_prompt,
    "max_tokens": 800,
    "temperature": 0.7,
    "top_p": 0.95,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "stream": True,  # Enable streaming response
}

# Headers
headers = {
    "Content-Type": "application/json",
    "api-key": api_key,  # Use 'api-key' instead of 'Authorization'
}

# Make the POST request with streaming
stream_string = "String: "
with requests.post(api_url, headers=headers, json=data, stream=True) as response:
    if response.status_code != 200:
        for line in response.iter_lines():
            if line is None:
                continue
            try:
                decoded_line = line.decode("utf-8").strip()
                if decoded_line.startswith("data: "):
                    decoded_line = decoded_line[len("data: "):]

                json_data = json.loads(decoded_line)
                # print("JSON:", json_data)
                if "choices" in json_data and len(json_data["choices"]) > 0 and "delta" in json_data["choices"][0]:
                    content = json_data["choices"][0]["delta"].get("content", "")
                    stream_string += content
                    print("CONTENT:", stream_string)
            except json.JSONDecodeError:
                continue
    else:
        print(f"Error: {response.status_code}, {response.text}")

print("\n\nFINAL: ", stream_string)


CONTENT: String: 
CONTENT: String: Artificial
CONTENT: String: Artificial intelligence
CONTENT: String: Artificial intelligence (
CONTENT: String: Artificial intelligence (AI
CONTENT: String: Artificial intelligence (AI)
CONTENT: String: Artificial intelligence (AI) is
CONTENT: String: Artificial intelligence (AI) is the
CONTENT: String: Artificial intelligence (AI) is the simulation
CONTENT: String: Artificial intelligence (AI) is the simulation of
CONTENT: String: Artificial intelligence (AI) is the simulation of human
CONTENT: String: Artificial intelligence (AI) is the simulation of human intelligence
CONTENT: String: Artificial intelligence (AI) is the simulation of human intelligence by
CONTENT: String: Artificial intelligence (AI) is the simulation of human intelligence by machines
CONTENT: String: Artificial intelligence (AI) is the simulation of human intelligence by machines,
CONTENT: String: Artificial intelligence (AI) is the simulation of human intelligence by machines, en